In [1]:
import subprocess
import shlex

In [2]:
import csb

In [4]:
def runHHblits( aln , name, path , outdir, db , iterations , ncores , runName , SS  , oa3m , verbose = False):
    if verbose == True:
        print( [aln , name, path , outdir, db , iterations , ncores , runName] )
    outhhr= outdir+name+runName+".hhr"
    args = path + ' -cpu '+ str(ncores) +' -d ' + db + ' -i ' + aln  +' -o '+ outhhr + ' -n ' + str(iterations) + ' -norealign -add_cons -M 50'
    if SS == True:
         args += ' -ssm 2 -ssw .5 '
    
    if oa3m == True:
        outa3m = outdir+name+runName+'.a3m'
        args += ' -oa3m ' + outa3m
    else: 
        outa3m = None
    if verbose == True:
        print(args)
    
    args = shlex.split(args)
    p = subprocess.Popen(args )
    return p , [outhhr,outa3m]


def runHHmake( aln , name, path = 'hhmake' , outdir='./', verbose = False, SS = False):
    if verbose == True:
        print( [aln , name, path , outdir] )
    outhhm= outdir+name+".hhm"
    args = path + ' -i '+  aln  +' -o '+ outhhm + ' -M 50'
    if SS == True:
        #todo : make ss prediction here
        pass
    args = shlex.split(args)
    p = subprocess.Popen(args )
    return p , [outhhm]


def hhrparse(hhr , coverage , proba ):
    profile = HHOutputParser(alignments=False).parse_file(hhr)
    qname = profile.query_name
    for hit in profile:
        proba = hit.probability
        i = hit.id

def query2PFAM_PDB(qaln, name , outdir= './' , path = 'hhblits' , verbose = False):
    uniclust = '/home/cactuskid13/mntpt/HHBLITsdb/uniclust30_2018_08/uniclust30_2018_08'
    pdb70 = '/home/cactuskid13/mntpt/HHBLITsdb/pdb70/pdb70'
    pfam = '/home/cactuskid13/mntpt/HHBLITsdb/pfam'

    p,ret = runHHmake(qaln, name)
    qaln = ret[0]
    
    db = uniclust
    iterations = 2
    ncores = 4
    
    runName = 'UNI'
    SS  = False
    oa3m = True
    
    p, ret = runHHblits( qaln ,name, path , outdir, db , iterations , ncores , runName , SS  , oa3m , verbose)
    p.wait()
    UNI_HHR = ret[0]
    '''
    #todo_add SS to model here
    
    db = pfam
    iterations = 1
    ncores = 1
    runName = 'PFAM'
    SS  = False
    oa3m = False
    qaln = ret[1]
    
    p, ret = runHHblits( qaln , name,  path , outdir, db , iterations , ncores , runName , SS  , oa3m , verbose)
    p.wait()
    PFAM_HHR = ret[1]
    
    
    db = pdb70
    iterations = 1
    ncores = 1
    runName = 'PDB'
    SS  = False
    oa3m = False
    
    p, ret = runHHblits( qaln ,name , path , outdir, db , iterations , ncores , runName , SS  , oa3m , verbose  )
    p.wait()
    PDB_HHR = ret[1]
    '''
    return UNI_HHR #, PDB_HHR , PFAM_HHR

def PFAM_HHR2GO(hhr):
    #map thru interpro
    
    pass

def PDB_HHR2GO(hhr):
    #map from pdb
    
    pass

def UNI_HHR2GO(hhr):
    #map from uniprot gaf
    
    #map to uniclust cluster rep and grab all identifiers
    
    #check if there are any annotations in cluster
    
    
    
    
    
    pass

In [5]:
#make pandas df of cath dom chains
import glob 
import pandas as pd

df = pd.read_csv('./pdb_chain_go.csv' , comment = '#' , error_bad_lines = False ,  warn_bad_lines = False )
print(df)



           PDB CHAIN SP_PRIMARY                WITH_STRING EVIDENCE  \
0         101m     A       IPRO         InterPro:IPR000971      IEA   
1         101m     A       IPRO         InterPro:IPR002335      IEA   
2         101m     A       IPRO         InterPro:IPR002335      IEA   
3         101m     A       IPRO         InterPro:IPR002335      IEA   
4         101m     A       IPRO         InterPro:IPR012292      IEA   
5         101m     A       IPRO         InterPro:IPR012292      IEA   
6         101m     A     P02185       UniProtKB-KW:KW-0479      IEA   
7         101m     A     P02185       UniProtKB-KW:KW-0561      IEA   
8         101m     A     P02185       UniProtKB-KW:KW-0561      IEA   
9         102l     A       IPRO         InterPro:IPR001165      IEA   
10        102l     A       IPRO         InterPro:IPR001165      IEA   
11        102l     A       IPRO         InterPro:IPR002196      IEA   
12        102l     A       IPRO         InterPro:IPR002196      IEA   
13    

In [7]:
#make a dataframe of uniprot noniea annotations
import pandas as pd
gaf = './goa_uniprot_all_noiea.gaf'
unigaf = pd.read_csv( gaf , comment = '!' , sep = '\t'  , header = None)
print(unigaf.head())

uniclust = '/home/cactuskid13/mntpt/HHBLITsdb/uniclust30_2018_08/uniclust30_2018_08.tsv'
dfclust = pd.read_csv( uniclust , comment = '!' , sep = '\t' , header = None )
print(dfclust.head())

          0           1           2    3           4               5    6   \
0  UniProtKB  A0A023FI02  A0A023FI02  NaN  GO:0044319  GO_REF:0000024  ISS   
1  UniProtKB  A0A023FI02  A0A023FI02  NaN  GO:0044319  GO_REF:0000024  ISS   
2  UniProtKB  A0A023FI02  A0A023FI02  NaN  GO:1902766  GO_REF:0000024  ISS   
3  UniProtKB  A0A023FI02  A0A023FI02  NaN  GO:1902766  GO_REF:0000024  ISS   
4  UniProtKB  A0A023FX47  A0A023FX47  NaN  GO:0044319  GO_REF:0000024  ISS   

                 7  8                                      9    10       11  \
0  UniProtKB:F1CG58  P  Putative ras protein family member ac  NaN  protein   
1  UniProtKB:G1MTS2  P  Putative ras protein family member ac  NaN  protein   
2  UniProtKB:F1CG58  P  Putative ras protein family member ac  NaN  protein   
3  UniProtKB:G1MTS2  P  Putative ras protein family member ac  NaN  protein   
4  UniProtKB:F1CG58  P  Putative ras protein family member ac  NaN  protein   

             12        13      14   15   16  
0   taxon:

In [ ]:
unigaf = unigaf.merge( dfclust , how   = 'left' , left_on = 1 , right_on =  1  )
print(unigaf)

In [ ]:
#make a branching tree of uniclust mapping
def lookup_clust(btree, ref):
    c = iter(ref)
    level = btree[next(c)]
    while len(level)>1:
        level = level[c]
    return level[c]

btree = {}

for l in uniclustin:
    clusterhead = l
    prots = []    
    for i,c in enumerate(clusterhead[0:-1]):
        if i == 0:
            if c not in btree:
                btree[c] = {}
            level = btree [c]
        else:
            if c not in level:
                level[c] = {}
            level[c] = {}
    else:
        #store refs in cluster head
        level[ clusterhead[0:-1]] = prots
        

In [ ]:
#make a dataframe of interpro domains mapped to GO terms
interpro_df = pd.read_csv('./interpro2go.txt' ,  sep = ';' , header = None, comment = '!' )
interpro_df['IPID'] = interpro_df[0].map( lambda x : x.split()[0].split(':')[1] )
pf_df = pd.read_csv('./ip2pfam.tsv' ,  sep = '\t' ,comment = '!' )
interpro_df = interpro_df.merge( pf_df , how='right' , right_on ='Accession' , left_on='IPID' )
interpro_df = interpro_df[ interpro_df[1].notnull() ]
#and map back to pfam

In [1]:
testaln = './Hap2HOG.aln.fasta'
import time
t0 = time.time()
query2PFAM_PDB(testaln, name='hap2' ,  outdir= '/home/cactuskid13/pyprofiler/pyprofiler/notebooks/cafa/' , path = 'hhblits' , verbose = True)
print( time.time() - t0)

NameError: name 'query2PFAM_PDB' is not defined

In [ ]:
#parse hhr files for each query and filter for nice hits


#map go from pdb chains and pfam domains


#output final pred

